In [1]:
# This class is for organizing and managing the training data for the Paradim Mask-rcnn classifier
# Its main goal is to query CasJobs for the location of all available training data and gather that data
# into a dataset folder containing dataset/[class]/[train or val]/.jpg-.json pairs
# This class will also organize the dataset into a k-fold cross-validation set, producing k sets of training/validation data
# This class would also potentially handle any img pre-processing

from SciServer import CasJobs
import os
import json
import shutil
import random
from SciServer import Jobs

create_table_query = '''
CREATE TABLE boron_carbide_training
(
Id INT NOT NULL IDENTITY PRIMARY KEY,
json_path varchar(150),
frame_path varchar(150),
class varchar(15)
)
'''
create_ballistics_query = '''
CREATE TABLE ballistics_training
(
Id INT NOT NULL IDENTITY PRIMARY KEY,
json_path varchar(150),
frame_path varchar(150),
class varchar(15)
)
'''

class TrainingDataManager():
    
    
    
    def getNextID(self):
        largest_ID_query = 'Select max(Id) from {0}'.format(self.training_data_table)
        id_df = CasJobs.executeQuery(sql=largest_ID_query, context=self.casjobs_context)
        if id_df['Column1'][0] == None:
            return 1
        else:
            return id_df['Column1'][0] + 1
        
    #given a directory full of .json/.jpg pairs, renames and moves into dataset directory, 
    #inserts record into CasJobs training data registry
    def insertTrainingDataCasJobs(self, current_class, folder_to_process):
        cur_ID = self.getNextID()
        for cur_json in os.listdir(folder_to_process):
            if '.json' in cur_json:
                new_json_fn = '{0}{1}.json'.format(current_class, cur_ID)
                new_frame_fn = '{0}{1}.jpg'.format(current_class, cur_ID)
        
                new_json_path = os.path.join(self.dataset_dir, current_class, new_json_fn)
                new_frame_path = os.path.join(self.dataset_dir, current_class, new_frame_fn)
        
                annotations = json.load(open(os.path.join(folder_to_process, cur_json)))
                cur_frame = annotations['imagePath']
                annotations['imagePath'] = new_frame_fn
        
                #copy edited json to new dataset location
                with open(new_json_path, 'w') as dumpfile:
                    json.dump(annotations, dumpfile)
        
                #copy frame to new dataset location
                shutil.copyfile(os.path.join(folder_to_process, cur_frame), new_frame_path)
        
                insert_query = '''insert into {3} values ('{0}', '{1}', '{2}') '''.format(new_json_path, new_frame_path, current_class, self.training_data_table)
                print(insert_query)
                CasJobs.executeQuery(sql=insert_query, context=self.casjobs_context)
        
                cur_ID = cur_ID + 1
            else:
                continue
 
    def getTrainingClasses(self):
        classes = []
        query = "select distinct class from {0}".format(self.training_data_table)
        class_df = CasJobs.executeQuery(sql=query, context=self.casjobs_context)
        for cur_class in class_df['class']:
            classes.append(cur_class)
        return classes
        
    def generateKFoldCrossVal(self, k):
        #create base directory
        if(os.path.isdir(self.cross_val_dir)):
            shutil.rmtree(self.cross_val_dir)
        os.makedirs(self.cross_val_dir)

            #in each cross-val set folder/[train or val]/[class]/[data]
        for i in range(1, k+1):
            cur_cross_val_dir = os.path.join(self.cross_val_dir, 'cross_val_' + str(i))                
            os.makedirs(cur_cross_val_dir)
            self.cross_val_dirs.append(cur_cross_val_dir)
            #make train and val dirs in each cross val dir
            os.makedirs(os.path.join(cur_cross_val_dir, 'train'))
            os.makedirs(os.path.join(cur_cross_val_dir, 'val'))
 
        
        for current_class in self.getTrainingClasses():
            #select all data elements in current class
            class_data_query = 'select json_path, frame_path from {0} where class = \'{1}\''.format(self.training_data_table, current_class)
            class_data_df = CasJobs.executeQuery(sql=class_data_query, context=self.casjobs_context)
            #we are using floor division. a few data elements may be left out but we will have equal size groups
            row_count = k * (class_data_df.shape[0] // k)
            print(row_count)
            random_ordering = list(range(0, row_count))
            random.shuffle(random_ordering)
            for cross_val_dir_index in range(0, k):
                cur_cross_val_dir = self.cross_val_dirs[cross_val_dir_index]
                #create class dir in val and train
                os.makedirs(os.path.join(cur_cross_val_dir, 'train', current_class))
                os.makedirs(os.path.join(cur_cross_val_dir, 'val', current_class))

                
                val_subset_index_range = range(int(cross_val_dir_index * (row_count / k)), int((cross_val_dir_index * (row_count / k)) + (row_count / k)))
                
                for i in range(0, row_count):
                    random_order_index = random_ordering[i]
                    cur_jpg = class_data_df['frame_path'][random_order_index]
                    cur_json = class_data_df['json_path'][random_order_index]
                    if i in val_subset_index_range:
                        shutil.copy(cur_jpg, os.path.join(cur_cross_val_dir, 'val', current_class))
                        shutil.copy(cur_json, os.path.join(cur_cross_val_dir, 'val', current_class))
                    else:
                        shutil.copy(cur_jpg, os.path.join(cur_cross_val_dir, 'train', current_class))
                        shutil.copy(cur_json, os.path.join(cur_cross_val_dir, 'train', current_class))                    
    
    def loadExistingCrossVal(self):
        for cur_dir in os.listdir(self.cross_val_dir):
            self.cross_val_dirs.append(os.path.join(self.cross_val_dir, cur_dir))
    def getCrossValidationDirs(self):
        return self.cross_val_dirs
    
    def __init__(self, casjobs_context='MyDB', k=10, dataset_dir='/home/idies/workspace/paradim_data/MRCNN_training_data/boron_carbide', training_data_table='boron_carbide_training'):
        
        self.casjobs_context=casjobs_context
        #dataset_directory structure: [dataset_directory]/[class]/jpg-json pairs
        self.dataset_dir = dataset_dir
        self.cross_val_dirs = []
        self.training_data_table = training_data_table
        self.k = k
        
        self.cross_val_dir = os.path.join(self.dataset_dir, '{0}_fold_cross_val'.format(k))


In [43]:
tdm = TrainingDataManager()
tdm.loadExistingCrossVal()
cross_val_dirs = tdm.getCrossValidationDirs()

steps = [500,1000]
#for hyperparam set
base_dir = '/home/idies/workspace/Temporary/ncarey/scratch/stepsPerEpoch'
#os.makedirs(base_dir)
for step in steps:
#we weill vary training length
    cur_step_dir = os.path.join(base_dir, str(step))
    os.makedirs(cur_step_dir)
    for i in range(0,len(cross_val_dirs)):
        
        cross_val_set = cross_val_dirs[i]
        #submit job
        #cross_val_set = '/home/idies/workspace/paradim_data/MRCNN_training_data/boron_carbide/10_fold_cross_val/cross_val_2'
        results_folder_path = os.path.join(cur_step_dir, '{0}_crossval_jobs/'.format(i))
        os.makedirs(results_folder_path)
        logs_folder_path = os.path.join(cur_step_dir, '{0}_crossval_logs/'.format(i))
        os.makedirs(logs_folder_path)
        results_file_path = os.path.join(cur_step_dir, '{0}_crossval_results.txt'.format(i))
        
        job_alias = 'testJob'
        job_cmd = '''pip install --user scikit-image && pip install --user keras==2.2.4 && pip install --user opencv-python-headless &&
python /home/idies/workspace/Storage/ncarey/persistent/PARADIM/furnace_ml/paradim.py train --dataset={0} --logs={1} --steps_per_epoch={3} --weights=imagenet &&
python /home/idies/workspace/Storage/ncarey/persistent/PARADIM/furnace_ml/paradim.py eval --dataset={0} --logs={1} --results={2} --steps_per_epoch={3} --weights=last'''.format(cross_val_set, logs_folder_path, results_file_path, step)
        job_id = Jobs.submitShellCommandJob(job_cmd, Jobs.getDockerComputeDomains()[6], 'TensorFlow', [{'name':'persistent'}, {'name':'scratch', 'needsWriteAccess':True}], [{'name':'PARADIM Data Collective', 'needsWriteAccess':True}], results_folder_path, job_alias)
        Jobs.waitForJob(job_id)

        with open(results_file_path, 'r') as resultsfile:
            print(resultsfile.read())
#save results file, process later?

    
#average scores 



Score: 0.9602950215339661, IoU: 0.9174945168365373, Class:3, GT: 3
Score: 0.9884047508239746, IoU: 0.9426286509040334, Class:3, GT: 3
Score: 0.9769288897514343, IoU: 0.9413186054539179, Class:3, GT: 3
Score: 0.9857629537582397, IoU: 0.870364010989011, Class:3, GT: 3
Score: 0.9861650466918945, IoU: 0.9184490449502459, Class:3, GT: 3
Score: 0.9885000586509705, IoU: 0.8892435729350738, Class:3, GT: 3
Score: 0.9798367619514465, IoU: 0.8373750982373415, Class:3, GT: 3
Score: 0.9825515151023865, IoU: 0.8671501755597767, Class:3, GT: 3
Score: 0.9760236740112305, IoU: 0.9190633170399, Class:3, GT: 3
Score: 0.9721881151199341, IoU: 0.9423280797229486, Class:3, GT: 3
Score: 0.9870703816413879, IoU: 0.9326437784285625, Class:3, GT: 3
Score: 0.9834983944892883, IoU: 0.9337253748897383, Class:3, GT: 3
Score: 0.9782619476318359, IoU: 0.9590163934426229, Class:3, GT: 3
Score: 0.9917357563972473, IoU: 0.8860302815235391, Class:3, GT: 3
Score: 0.9036847352981567, IoU: 0.9084084084084084, Class:3, GT: 3

Score: 0.9948758482933044, IoU: 0.929514963880289, Class:2, GT: 2
Score: 0.9881133437156677, IoU: 0.9341646412342809, Class:2, GT: 2
Score: 0.9967970252037048, IoU: 0.9274015898860263, Class:2, GT: 2
Score: 0.9949565529823303, IoU: 0.9304852913621885, Class:2, GT: 2
Score: 0.9937289953231812, IoU: 0.9018699755330304, Class:2, GT: 2
Score: 0.9963937401771545, IoU: 0.891583452211127, Class:2, GT: 2
Score: 0.9956477284431458, IoU: 0.9509577775549093, Class:2, GT: 2
Score: 0.9937601685523987, IoU: 0.865552506859566, Class:2, GT: 2
Score: 0.9955755472183228, IoU: 0.9151709974104831, Class:2, GT: 2
Score: 0.9982055425643921, IoU: 0.8623202487368831, Class:2, GT: 2
Score: 0.9967169165611267, IoU: 0.9283884847863948, Class:2, GT: 2
Score: 0.9965516328811646, IoU: 0.8959351283917247, Class:2, GT: 2
Score: 0.9949994087219238, IoU: 0.8717897069191258, Class:2, GT: 2
Score: 0.9985254406929016, IoU: 0.921451317186143, Class:2, GT: 2
Score: 0.9980813264846802, IoU: 0.9153981567774672, Class:2, GT: 2

Score: 0.9925332069396973, IoU: 0.8906789413118527, Class:3, GT: 3
Score: 0.9991232752799988, IoU: 0.9457028834804114, Class:3, GT: 3
Score: 0.9989872574806213, IoU: 0.9400455062571104, Class:3, GT: 3
Score: 0.9993471503257751, IoU: 0.9410341529285996, Class:3, GT: 3
Score: 0.9989978671073914, IoU: 0.9280305155101812, Class:3, GT: 3
Score: 0.9995378255844116, IoU: 0.9390545016561277, Class:3, GT: 3
Score: 0.9995526671409607, IoU: 0.9142440939894864, Class:3, GT: 3
Score: 0.9994408488273621, IoU: 0.9608660877513712, Class:3, GT: 3
Score: 0.9988225102424622, IoU: 0.9636322049405307, Class:3, GT: 3
Score: 0.9993657469749451, IoU: 0.9346445630795167, Class:3, GT: 3
Score: 0.999552309513092, IoU: 0.951628023248547, Class:3, GT: 3
Score: 0.9994338154792786, IoU: 0.9262525275219052, Class:3, GT: 3
Score: 0.999657154083252, IoU: 0.954433062522077, Class:3, GT: 3
Score: 0.9996702671051025, IoU: 0.9686925621860066, Class:3, GT: 3
Score: 0.999427080154419, IoU: 0.8915072482954921, Class:3, GT: 3


KeyboardInterrupt: 

In [35]:
#!python /home/idies/workspace/Storage/ncarey/persistent/PARADIM/furnace_ml/paradim.py eval --dataset='/home/idies/workspace/paradim_data/MRCNN_training_data/boron_carbide/10_fold_cross_val/cross_val_1' --logs='/home/idies/workspace/Temporary/ncarey/scratch/logs' --results='/home/idies/workspace/Temporary/ncarey/scratch/results.txt' --weights=last

Using TensorFlow backend.
Weights:  last
Dataset:  /home/idies/workspace/paradim_data/MRCNN_training_data/boron_carbide/10_fold_cross_val/cross_val_1
Logs:  /home/idies/workspace/Temporary/ncarey/scratch/logs

Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.3
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  576
IMAGE_META_SIZE                16
IMAGE_MIN_DIM                  432
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [576 576   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.00

In [14]:
Jobs.getDockerComputeDomains()[6]['volumes']

[{'description': 'PARADIM Data Collective',
  'id': 20216,
  'name': 'PARADIM Data Collective',
  'publisherDID': '149',
  'racmUUID': '724cb4dc-10f7-4baa-b447-4344bcbedb78',
  'writable': True},
 {'description': 'Getting Started',
  'id': 20229,
  'name': 'Getting Started',
  'publisherDID': '158',
  'racmUUID': '8e7ebf30-3cd7-4ea3-b070-07d3ac06c95c',
  'writable': False}]

In [ ]:
CREATE TABLE paradim_config
(
Id INT NOT NULL IDENTITY PRIMARY KEY,
name varchar(20),
steps_per_epoch int,
validation_steps int,
FPN_CLASSIF_FC_LAYERS_SIZE int,
TOP_DOWN_PYRAMID_SIZE int,
NUM_CLASSES int,
RPN_ANCHOR_SCALES varchar(150),
RPN_ANCHOR_RATIOS varchar(150),
RPN_ANCHOR_STRIDE int,
RPN_NMS_THRESHOLD float,
RPN_TRAIN_ANCHORS_PER_IMAGE int,
PRE_NMS_LIMIT int,
POST_NMS_ROIS_TRAINING int,
POST_NMS_ROIS_INFERENCE int,
    
)
'''